In [1]:
%load_ext autoreload
%autoreload 2
import time
import os
import seaborn as sns
import numpy as np
import pandas as pd
import sys

from mimas.tools.spectral_file.extract_ms1_feature import process_mzml_file, extract_ms2_spectra, find_features, load_mzml_data
import logging
from toolsets.spectra_operations import entropy_similarity_default
from toolsets.std_list_prep import complete_adducts, complete_formula
# from toolsets.API_gets import complete_smiles
from toolsets.search import string_search, num_search
import toolsets.spectra_operations as so
from tqdm import tqdm
# from toolsets.features_by_alphapept import find_features_alphapept
# std_list_path= "/Users/fanzhoukong/Documents/GitHub/Libgen_data/GT"
from toolsets.file_io import read_in_alphapept
source_dir = "/Users/fanzhoukong/Documents/GitHub/Libgen_data/GT"
# std_list = pd.read_csv(os.path.join(source_dir, "enlongated_list.csv"))
# enzyme_list = pd.read_csv(os.path.join(source_dir, "enzyme_list_cor.csv"))
# feature_dir = os.path.join(source_dir, "features")
from toolsets.precursor_matching import precursor_matching_alphapept, GT_precursor_matching
from toolsets.precursor_matching import GT_precursor_matching

# testing for missing files

In [3]:
features_dir = '/Users/fanzhoukong/Documents/GitHub/Libgen_data/GT/features'
bad_files = []
missing_files = []
negative_files = []
for mix in (std_list['Mix'].unique()):
    for enzyme in tqdm(enzyme_list['List of enzymes'].unique()):
        filename =enzyme+"_"+"Mix"+"_"+str(mix)
        feature_path = os.path.join(feature_dir, filename+".csv")
        if os.path.exists(feature_path) ==True:
            try:
                features = read_in_alphapept(feature_path)
                if features['charge'].min()<0:
                    negative_files.append(filename)
            except:
                bad_files.append(filename)
                # print("the file is bad %s" %(filename) )
        else:
            missing_files.append(filename)
            # print("the file doesnt exist")


100%|██████████| 109/109 [00:00<00:00, 256.42it/s]


# test mzml files

In [26]:
# negative_files.to_csv(os.path.join(source_dir, "negatively_charged_files.csv"), index= True)

In [62]:
from mimas.tools.spectral_file.extract_ms1_feature import load_mzml_data

In [63]:
test_mzml = load_mzml_data('/Users/fanzhoukong/Documents/GitHub/Libgen_data/casmi/mzml/A_M3_negPFP_03.mzml')

In [80]:
test_mzml.keys()

dict_keys(['scan_list_ms1', 'rt_list_ms1', 'mass_list_ms1', 'int_list_ms1', 'ms_list_ms1', 'scan_list_ms2', 'rt_list_ms2', 'mass_list_ms2', 'int_list_ms2', 'ms_list_ms2', 'prec_mass_list2', 'mono_mzs2', 'charge2', 'select_windows_ms2', 'indices_ms1'])

In [81]:
len(test_mzml['ms_list_ms1'])

4183

In [83]:
test_mzml['ms_list_ms1'][1]

1

In [65]:
test_mzml_df = pd.DataFrame.from_dict(test_mzml)

ValueError: Per-column arrays must each be 1-dimensional

# GT matching

In [3]:
from toolsets.precursor_matching import GT_precursor_matching
matched =GT_precursor_matching(std_list, enzyme_list, feature_dir, adducts=std_list.columns[-4:], step=0.003, ppm = False)

i am in new method


100%|██████████| 15/15 [08:34<00:00, 34.28s/it]


In [5]:
matched.to_csv(os.path.join(source_dir, 'matched_updated_202209121.csv'), index= False)

# some post processing

In [3]:
matched = pd.read_csv(os.path.join(source_dir, 'matched_updated_202209121.csv'))

In [17]:
rt_list = pd.read_csv(os.path.join(source_dir, 'RT_list.csv'))
rt_list["RT"] = pd.to_numeric(rt_list["RT"], errors='coerce')

rt_list_checked = rt_list.dropna(subset=['RT'])

In [18]:
rt_list_checked['RT'].isna().sum()

0

In [23]:
matched.head()

,scan_number,precursor_mz,charge,ms1_intensity_ratio,ms1_precursor_intensity,retention_time,peaks,ms1_index,query_idx,reference_Mix,reference_Substrate Name,reference_Formula,reference_CAS number,reference_inchikey,reference_smiles,reference_adduct,reference_precursor_mz,reference_mix_enzyme
0,3037.0,462.248047,1.0,1.000000,2642259.0,5.930206,51.6996078\t0.00643718708\n52.0238495\t0.00572...,610.0,2426.0,1.0,Strophanthidinic Acid Lactone Acetate,C25H32O7,No CAS#,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,[M+NH4]+,462.2486,E5_Mix_1
1,3096.0,462.248266,1.0,1.000000,932831.0,6.047565,56.0496788\t0.0164161809\n57.0337524\t0.136146...,622.0,2473.0,1.0,Strophanthidinic Acid Lactone Acetate,C25H32O7,No CAS#,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,[M+NH4]+,462.2486,E5_Mix_1
2,3311.0,462.248285,1.0,1.000000,4048701.0,6.472063,55.0395012\t0.0112954155\n56.0497665\t0.020464...,665.0,2645.0,1.0,Strophanthidinic Acid Lactone Acetate,C25H32O7,No CAS#,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,[M+NH4]+,462.2486,E5_Mix_1
3,3558.0,462.248152,1.0,0.972492,9181763.0,6.957039,53.0386429\t0.00317348423\n55.0431709\t0.01122...,714.0,2843.0,1.0,Strophanthidinic Acid Lactone Acetate,C25H32O7,No CAS#,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,[M+NH4]+,462.2486,E5_Mix_1
4,4262.0,607.274555,1.0,0.774424,161785.0,8.343247,53.4098587\t0.0547753833\n70.065239\t0.0801916...,855.0,3406.0,1.0,Strophanthidinic Acid Lactone Acetate_glycoside,C25H32O7,No CAS#,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,[M+H]+,607.2754,E5_Mix_1


In [24]:
rt_list_checked.head()

,Mix,Substrate Name,CAS number selected,CAS to Inchi (CTS Proxy) for MoNA,For all (CAS or no CAS) Name to Smiles by pubchem,RT
13,1,3-Amino-Beta-Pinene,No CAS#,ZEKALTOSWFXTCN-UHFFFAOYSA-N,CC1(C2CC1C(=C)C(C2)N)C,5.11
14,1,3-Hydroxytyramine,62-31-7,VYFYYTLLBUKUHU-UHFFFAOYSA-N,C1=CC(=C(C=C1CCN)O)O,1.13
15,1,Lupinine,486-70-4,HDVAWXXJVMJBAR-VHSXEESVSA-N,C1CCN2CCC[C@H]([C@H]2C1)CO,1.70
16,1,Methyl Orsellinate,3187-58-4,NCCWCZLEACWJIN-UHFFFAOYSA-N,CC1=CC(=CC(=C1C(=O)OC)O)O,7.40
17,1,Dihydrojasmonic Acid,98674-52-3,PQEYTAGBXNEUQL-UHFFFAOYSA-N,CCCCCC1C(CCC1=O)CC(=O)O,8.20


In [64]:
RT_matched = pd.DataFrame()
RT_unmatched = pd.DataFrame()
matched_rt = []
unmatched_rt = []
for index, row in tqdm(matched.iterrows(), total=len(matched)):
    if row['reference_Substrate Name'].endswith('_glycoside')==False:
        data_temp = string_search(rt_list_checked, 'Substrate Name', row['reference_Substrate Name'])
        if len(data_temp)>0:
            rt = data_temp.iloc[0]['RT']
            if abs(row['retention_time']-rt)<=20/60:
                # matched_rt.append('RT not found in list')
                RT_matched=RT_matched.append(row)
                matched_rt.append(rt)
            else:
                RT_unmatched=RT_unmatched.append(row)
                unmatched_rt.append(rt)
            # data_temp.
            # pass
        else:
            RT_matched=RT_matched.append(row)
            matched_rt.append('RT not found in list')
    else:
        RT_matched=RT_matched.append(row)
        matched_rt.append('RT matched for glycosides')




100%|██████████| 275623/275623 [23:01<00:00, 199.53it/s]


In [76]:
RT_matched.to_csv(os.path.join(source_dir, 'matched_by_rt.csv'), index = False)

In [77]:
RT_unmatched.to_csv(os.path.join(source_dir, 'matched_not_by_rt.csv'), index = False)

In [74]:
RT_unmatched.head()

,scan_number,precursor_mz,charge,ms1_intensity_ratio,ms1_precursor_intensity,retention_time,peaks,ms1_index,query_idx,reference_Mix,reference_Substrate Name,reference_Formula,reference_CAS number,reference_inchikey,reference_smiles,reference_adduct,reference_precursor_mz,reference_mix_enzyme,refernce_RT
0,3037.0,462.248047,1.0,1.000000,2642259.0,5.930206,51.6996078\t0.00643718708\n52.0238495\t0.00572...,610.0,2426.0,1.0,Strophanthidinic Acid Lactone Acetate,C25H32O7,No CAS#,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,[M+NH4]+,462.2486,E5_Mix_1,8.69
1,3096.0,462.248266,1.0,1.000000,932831.0,6.047565,56.0496788\t0.0164161809\n57.0337524\t0.136146...,622.0,2473.0,1.0,Strophanthidinic Acid Lactone Acetate,C25H32O7,No CAS#,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,[M+NH4]+,462.2486,E5_Mix_1,8.69
2,3311.0,462.248285,1.0,1.000000,4048701.0,6.472063,55.0395012\t0.0112954155\n56.0497665\t0.020464...,665.0,2645.0,1.0,Strophanthidinic Acid Lactone Acetate,C25H32O7,No CAS#,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,[M+NH4]+,462.2486,E5_Mix_1,8.69
3,3558.0,462.248152,1.0,0.972492,9181763.0,6.957039,53.0386429\t0.00317348423\n55.0431709\t0.01122...,714.0,2843.0,1.0,Strophanthidinic Acid Lactone Acetate,C25H32O7,No CAS#,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,[M+NH4]+,462.2486,E5_Mix_1,8.69
7,5801.0,235.169187,1.0,0.812239,3155052.0,11.392168,53.002438\t0.019678773\n55.054535\t0.01263231\...,1163.0,4637.0,1.0,Punctaporonin B,C15H24O3,93697-36-0,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,[M-H2O+H]+,235.1693,E5_Mix_1,6.84


In [16]:
matched.to_csv(os.path.join(source_dir, "matched.csv"), index = False)

In [11]:
substrate_list = pd.read_csv(os.path.join(source_dir,"substrate_list.csv" ))

In [12]:
substrate_list

,Mix,Substrate Name,Formula,CAS number,inchikey,smiles,[M+H]+,[M+NH4]+,[M+Na]+,[M-H2O+H]+
0,1,Strophanthidinic Acid Lactone Acetate,C25H32O7,No CAS#,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,445.2221,462.2486,467.2040,427.2115
1,1,Glutamine (L),C5H10N2O3,56-85-9,ZDXPYRJPNDTMRX-VKHMYHEASA-N,C(CC(=O)N)[C@@H](C(=O)O)N,147.0764,164.1030,169.0584,129.0659
2,1,Punctaporonin B,C15H24O3,93697-36-0,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,253.1798,270.2064,275.1618,235.1693
3,1,Lupinine,C10H19NO,486-70-4,HDVAWXXJVMJBAR-VHSXEESVSA-N,C1CCN2CCC[C@H]([C@H]2C1)CO,170.1539,187.1805,192.1359,152.1434
4,1,Glycocholic Acid,C26H43NO6,475-31-0,RFDAIACWWDREDC-FRVQLJSFSA-N,C[C@H](CCC(=O)NCC(=O)O)[C@H]1CC[C@@H]2[C@@]1([...,466.3163,483.3429,488.2983,448.3057
...,...,...,...,...,...,...,...,...,...,...
588,15,Isomaltulose,C12H22O11,13718-94-0,PVXPPJIGRGXGCY-TZLCEDOOSA-N,C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)OC[C@@H]2...,343.1235,360.1500,365.1054,325.1129
589,15,Deoxyuridine,C9H12N2O5,951-78-0,MXHRCPNRJAMMIM-SHYZEUOFSA-N,C1[C@@H]([C@H](O[C@H]1N2C=CC(=O)NC2=O)CO)O,229.0819,246.1084,251.0638,211.0713
590,15,Quinine,C20H24N2O2,130-95-0,LOUPRKONTZGTKE-WZBLMQSHSA-N,COC1=CC2=C(C=CN=C2C=C1)[C@H]([C@@H]3C[C@@H]4CC...,325.1911,342.2176,347.1730,307.1805
591,15,Chlorogenic Acid,C16H18O9,327-97-9,CWVRJTMFETXNAD-JUHZACGLSA-N,C1[C@H]([C@H]([C@@H](C[C@@]1(C(=O)O)O)OC(=O)/C...,355.1024,372.1289,377.0843,337.0918


In [14]:
import re
def find_files(base, pattern):
    '''Return list of files matching pattern in base folder.'''
    return [filename for filename in os.listdir(base) if re.search(pattern, filename, re.IGNORECASE)]

In [17]:
mix_dir = '/Users/fanzhoukong/Documents/GitHub/Libgen_data/GT/mix_folder'
from toolsets.precursor_matching import precursor_matching_alphapept
substrate_matched = pd.DataFrame()
for mix in tqdm(substrate_list['Mix'].unique()):
    # break
    try:
        feature_filename = find_files(mix_dir, f"{'Mix_'+str(mix)}.csv")
        features_temp = read_in_alphapept(os.path.join(mix_dir, feature_filename[0]))
        substrate_matched_temp = precursor_matching_alphapept(features_temp, std_list, mix_label=mix, adducts= substrate_list.columns[-4:], step=0.003, ppm=False)
        substrate_matched = pd.concat([substrate_matched, substrate_matched_temp], axis=0)
    except:
        print("i do not found file for mix %s" %str(mix))
    # print((feature_filename[0]))


  # CSH_Mix_2.csv


100%|██████████| 15/15 [00:03<00:00,  4.67it/s]


In [18]:
substrate_matched

,scan_number,precursor_mz,charge,ms1_intensity_ratio,ms1_precursor_intensity,retention_time,peaks,ms1_index,query_idx,reference_Mix,reference_Name,reference_Formula,reference_inchikey,reference_smiles,reference_adduct,reference_precursor_mz
1390,5642,427.211479,1,0.922873,1495755.0,11.155714,53.5735168\t0.00877639279\n60.1089172\t0.00829...,1129,4512,1,Strophanthidinic Acid Lactone Acetate,C25H32O7,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,[M-H2O+H]+,427.2115
677,3501,235.169055,1,0.905345,11106662.0,6.898704,50.1851387\t0.00321989087\n51.0233536\t0.00314...,701,2799,1,Punctaporonin B,C15H24O3,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,[M-H2O+H]+,235.1693
694,3563,235.169182,1,0.651131,2149314.0,7.019838,50.5072937\t0.00643686485\n55.0444412\t0.03300...,713,2849,1,Punctaporonin B,C15H24O3,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,[M-H2O+H]+,235.1693
109,965,170.153793,1,0.964765,674355712.0,1.887554,70.065094\t0.00733170193\n81.0696945\t0.007615...,193,770,1,Lupinine,C10H19NO,HDVAWXXJVMJBAR-VHSXEESVSA-N,C1CCN2CCC[C@H]([C@H]2C1)CO,[M+H]+,170.1539
111,1085,170.153781,1,0.966453,349071584.0,2.123019,67.054291\t0.0074617863\n84.080482\t0.01529860...,217,866,1,Lupinine,C10H19NO,HDVAWXXJVMJBAR-VHSXEESVSA-N,C1CCN2CCC[C@H]([C@H]2C1)CO,[M+H]+,170.1539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,1243,341.108363,1,0.697549,109407.0,7.045587,59.0662956\t0.167543039\n89.3574219\t0.2353442...,1127,115,15,Lactobionic Acid,C12H22O12,JYTUSYBCFIZPBE-AMTLMPIISA-N,C([C@@H]1[C@@H]([C@@H]([C@H]([C@@H](O1)O[C@H](...,[M-H2O+H]+,341.1078
28,1194,411.090546,1,0.788332,308502.0,6.802072,63.259926\t0.079237752\n63.94931\t0.089768991\...,1089,104,15,7-DESHYDROXYPYROGALLIN-4-CARBOXYLIC ACID_glyco...,C12H8O6,XCSLJQFPTAZKDV-UHFFFAOYSA-N,C1=CC(=C(C2=C(C(=O)C=C(C=C21)C(=O)O)O)O)O,[M+H]+,411.0927
9,236,343.126038,1,0.601625,312877.0,1.310034,55.04211\t0.025685305\n56.835411\t0.025831141\...,208,27,15,Isomaltulose,C12H22O11,PVXPPJIGRGXGCY-TZLCEDOOSA-N,C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)OC[C@@H]2...,[M+H]+,343.1235
24,1142,414.159893,1,0.979704,2790103.0,6.539994,79.0542603\t0.0220709741\n80.0495834\t0.022314...,1048,93,15,2'-deoxyadenosine_glycoside,C10H13N5O3,OLXZPDWKRNYJJZ-FSDSQADBSA-N,C1[C@H]([C@H](O[C@H]1N2C=NC3=C(N=CN=C32)N)CO)O,[M+H]+,414.1624


In [19]:
substrate_matched.to_csv(os.path.join(source_dir, 'substrate_matched_20220939.csv'), index = False)

In [13]:
rt_list_checked

,Mix,Substrate Name,CAS number selected,CAS to Inchi (CTS Proxy) for MoNA,For all (CAS or no CAS) Name to Smiles by pubchem,RT
13,1,3-Amino-Beta-Pinene,No CAS#,ZEKALTOSWFXTCN-UHFFFAOYSA-N,CC1(C2CC1C(=C)C(C2)N)C,5.11
14,1,3-Hydroxytyramine,62-31-7,VYFYYTLLBUKUHU-UHFFFAOYSA-N,C1=CC(=C(C=C1CCN)O)O,1.13
15,1,Lupinine,486-70-4,HDVAWXXJVMJBAR-VHSXEESVSA-N,C1CCN2CCC[C@H]([C@H]2C1)CO,1.70
16,1,Methyl Orsellinate,3187-58-4,NCCWCZLEACWJIN-UHFFFAOYSA-N,CC1=CC(=CC(=C1C(=O)OC)O)O,7.40
17,1,Dihydrojasmonic Acid,98674-52-3,PQEYTAGBXNEUQL-UHFFFAOYSA-N,CCCCCC1C(CCC1=O)CC(=O)O,8.20
...,...,...,...,...,...,...
589,15,Isomaltulose,13718-94-0,PVXPPJIGRGXGCY-TZLCEDOOSA-N,C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)OC[C@@H]2...,6.73
590,15,Deoxyuridine,951-78-0,MXHRCPNRJAMMIM-SHYZEUOFSA-N,C1[C@@H]([C@H](O[C@H]1N2C=CC(=O)NC2=O)CO)O,1.68
591,15,Quinine,130-95-0,LOUPRKONTZGTKE-WZBLMQSHSA-N,COC1=CC2=C(C=CN=C2C=C1)[C@H]([C@@H]3C[C@@H]4CC...,2.07
592,15,Chlorogenic Acid,327-97-9,CWVRJTMFETXNAD-JUHZACGLSA-N,C1[C@H]([C@H]([C@@H](C[C@@]1(C(=O)O)O)OC(=O)/C...,4.08


# RT matching for substrates

In [22]:
RT_matched = pd.DataFrame()
RT_unmatched = pd.DataFrame()
matched_rt = []
unmatched_rt = []
for index, row in tqdm(substrate_matched.iterrows(), total=len(substrate_matched)):
    data_temp = string_search(rt_list_checked, 'Substrate Name', row['reference_Name'])
    if len(data_temp)>0:
        rt = data_temp.iloc[0]['RT']
        if abs(row['retention_time']-rt)<=20/60:
            RT_matched=RT_matched.append(row)
            matched_rt.append(rt)
        else:
            RT_unmatched=RT_unmatched.append(row)
            unmatched_rt.append(rt)
    else:
        RT_matched=RT_matched.append(row)
        matched_rt.append('RT not found in list')


100%|██████████| 2381/2381 [00:02<00:00, 916.12it/s]


In [23]:
RT_matched['reference_RT']=matched_rt
RT_unmatched['reference_RT']=unmatched_rt

In [24]:
RT_matched

,scan_number,precursor_mz,charge,ms1_intensity_ratio,ms1_precursor_intensity,retention_time,peaks,ms1_index,query_idx,reference_Mix,reference_Name,reference_Formula,reference_inchikey,reference_smiles,reference_adduct,reference_precursor_mz,reference_RT
677,3501,235.169055,1,0.905345,1.110666e+07,6.898704,50.1851387\t0.00321989087\n51.0233536\t0.00314...,701,2799,1,Punctaporonin B,C15H24O3,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,[M-H2O+H]+,235.1693,6.84
694,3563,235.169182,1,0.651131,2.149314e+06,7.019838,50.5072937\t0.00643686485\n55.0444412\t0.03300...,713,2849,1,Punctaporonin B,C15H24O3,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,[M-H2O+H]+,235.1693,6.84
109,965,170.153793,1,0.964765,6.743557e+08,1.887554,70.065094\t0.00733170193\n81.0696945\t0.007615...,193,770,1,Lupinine,C10H19NO,HDVAWXXJVMJBAR-VHSXEESVSA-N,C1CCN2CCC[C@H]([C@H]2C1)CO,[M+H]+,170.1539,1.7
1412,5697,349.234929,1,0.959309,4.613596e+06,11.264991,53.0329475\t0.106125951\n69.0699387\t0.1171013...,1140,4556,1,Lupinine_glycoside,C10H19NO,HDVAWXXJVMJBAR-VHSXEESVSA-N,C1CCN2CCC[C@H]([C@H]2C1)CO,[M+NH4]+,349.2338,RT not found in list
870,4070,466.315908,1,0.970606,1.003590e+09,8.021085,53.0387917\t0.00252936361\n55.0492134\t0.03133...,814,3254,1,Glycocholic Acid,C26H43NO6,RFDAIACWWDREDC-FRVQLJSFSA-N,C[C@H](CCC(=O)NCC(=O)O)[C@H]1CC[C@@H]2[C@@]1([...,[M+H]+,466.3163,8.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,578,620.217651,1,0.599324,8.373400e+04,3.134361,57.0701065\t0.0832991451\n79.1539307\t0.051101...,500,77,15,Amygdalin_glycoside,C20H27NO11,XUCIJNAGGSZNQT-JHSLDZJXSA-N,C1=CC=C(C=C1)[C@H](C#N)O[C@H]2[C@@H]([C@H]([C@...,[M+H]+,620.2190,RT not found in list
31,1243,341.108363,1,0.697549,1.094070e+05,7.045587,59.0662956\t0.167543039\n89.3574219\t0.2353442...,1127,115,15,Lactobionic Acid,C12H22O12,JYTUSYBCFIZPBE-AMTLMPIISA-N,C([C@@H]1[C@@H]([C@@H]([C@H]([C@@H](O1)O[C@H](...,[M-H2O+H]+,341.1078,RT not found in list
28,1194,411.090546,1,0.788332,3.085020e+05,6.802072,63.259926\t0.079237752\n63.94931\t0.089768991\...,1089,104,15,7-DESHYDROXYPYROGALLIN-4-CARBOXYLIC ACID_glyco...,C12H8O6,XCSLJQFPTAZKDV-UHFFFAOYSA-N,C1=CC(=C(C2=C(C(=O)C=C(C=C21)C(=O)O)O)O)O,[M+H]+,411.0927,RT not found in list
24,1142,414.159893,1,0.979704,2.790103e+06,6.539994,79.0542603\t0.0220709741\n80.0495834\t0.022314...,1048,93,15,2'-deoxyadenosine_glycoside,C10H13N5O3,OLXZPDWKRNYJJZ-FSDSQADBSA-N,C1[C@H]([C@H](O[C@H]1N2C=NC3=C(N=CN=C32)N)CO)O,[M+H]+,414.1624,RT not found in list


In [8]:
RT_matched.to_csv(os.path.join(source_dir, "substrate_matched_rt.csv"), index= False)
RT_unmatched.to_csv(os.path.join(source_dir, "substrate_unmatched_rt.csv"), index= False)

In [9]:
RT_matched.head()

,scan_number,precursor_mz,charge,ms1_intensity_ratio,ms1_precursor_intensity,retention_time,peaks,ms1_index,query_idx,reference_Mix,reference_Substrate Name,reference_Formula,reference_CAS number,reference_inchikey,reference_smiles,reference_adduct,reference_precursor_mz,reference_mix_enzyme,refernce_RT
0,4262.0,607.274555,1.0,0.774424,161785.0,8.343247,53.4098587\t0.0547753833\n70.065239\t0.0801916...,855.0,3406.0,1.0,Strophanthidinic Acid Lactone Acetate_glycoside,C25H32O7,No CAS#,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,[M+H]+,607.2754,E5_Mix_1,RT matched for glycosides
1,4540.0,309.130771,1.0,0.992883,12260732.0,8.896537,51.104725\t0.12610184\n59.190243\t0.1361344\n6...,910.0,3628.0,1.0,Glutamine (L)_glycoside,C5H10N2O3,56-85-9,ZDXPYRJPNDTMRX-VKHMYHEASA-N,C(CC(=O)N)[C@@H](C(=O)O)N,[M+H]+,309.1297,E5_Mix_1,RT matched for glycosides
2,3506.0,235.169062,1.0,0.985012,64803040.0,6.855780,52.6032257\t0.00468503637\n53.0388222\t0.01121...,704.0,2801.0,1.0,Punctaporonin B,C15H24O3,93697-36-0,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,[M-H2O+H]+,235.1693,E5_Mix_1,6.84
3,2932.0,415.235046,1.0,0.630249,196358.0,5.723194,50.799168\t0.03342418\n51.667332\t0.033813868\...,589.0,2342.0,1.0,Punctaporonin B_glycoside,C15H24O3,93697-36-0,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,[M+H]+,415.2331,E5_Mix_1,RT matched for glycosides
4,4172.0,415.234833,1.0,0.043016,82492.0,8.165338,52.5774269\t0.00837360416\n52.8593597\t0.00794...,837.0,3334.0,1.0,Punctaporonin B_glycoside,C15H24O3,93697-36-0,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,[M+H]+,415.2331,E5_Mix_1,RT matched for glycosides


In [6]:
matched = pd.read_csv(os.path.join(source_dir, 'matched_updated_202209121.csv'))
RT_matched= pd.read_csv(os.path.join(source_dir, "matched_by_rt.csv"))
RT_unmatched=pd.read_csv(os.path.join(source_dir, "matched_not_by_rt.csv"))

In [4]:
std_list.head()

,Mix,Name,Formula,inchikey,smiles,[M+H]+,[M+NH4]+,[M+Na]+,[M-H2O+H]+
0,1,Strophanthidinic Acid Lactone Acetate,C25H32O7,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,445.2221,462.2486,467.2040,427.2115
1,1,Strophanthidinic Acid Lactone Acetate_glycoside,C25H32O7,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,607.2754,624.3019,629.2573,589.2648
2,1,Strophanthidinic Acid Lactone Acetate_diglycoside,C25H32O7,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,769.3287,786.3552,791.3106,751.3181
3,1,Glutamine (L),C5H10N2O3,ZDXPYRJPNDTMRX-VKHMYHEASA-N,C(CC(=O)N)[C@@H](C(=O)O)N,147.0764,164.1030,169.0584,129.0659
4,1,Glutamine (L)_glycoside,C5H10N2O3,ZDXPYRJPNDTMRX-VKHMYHEASA-N,C(CC(=O)N)[C@@H](C(=O)O)N,309.1297,326.1563,331.1117,291.1192


In [5]:
string_search(std_list, 'Name','Kaempferol')

,Mix,Name,Formula,inchikey,smiles,[M+H]+,[M+NH4]+,[M+Na]+,[M-H2O+H]+
873,8,Kaempferol,C15H10O6,IYRMWMYZSQPJKC-UHFFFAOYSA-N,C1=CC(=CC=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O,287.055,304.0816,309.037,269.0444


In [6]:
string_search(std_list, 'inchikey','IYRMWMYZSQPJKC-UHFFFAOYSA-N')

,Mix,Name,Formula,inchikey,smiles,[M+H]+,[M+NH4]+,[M+Na]+,[M-H2O+H]+
873,8,Kaempferol,C15H10O6,IYRMWMYZSQPJKC-UHFFFAOYSA-N,C1=CC(=CC=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O,287.0550,304.0816,309.0370,269.0444
874,8,Kaempferol_glycoside,C15H10O6,IYRMWMYZSQPJKC-UHFFFAOYSA-N,C1=CC(=CC=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O,449.1083,466.1349,471.0903,431.0977
875,8,Kaempferol_diglycoside,C15H10O6,IYRMWMYZSQPJKC-UHFFFAOYSA-N,C1=CC(=CC=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O,611.1616,628.1882,633.1436,593.1510


In [7]:
len(RT_matched)+len(RT_unmatched)

275623

In [5]:
len(matched)

275623

# matching for NCs

In [66]:
from toolsets.helpers import check_for_missing_compounds

In [8]:
feature_filename

['HILIC_pos_03282022_Pet28GFP_Mix_15.csv']

In [34]:
substrate_list = pd.read_csv(os.path.join(source_dir, "substrate_list.csv"))

In [39]:
std_list.columns[-4:]

Index(['[M+H]+', '[M+NH4]+', '[M+Na]+', '[M-H2O+H]+'], dtype='object')

In [40]:
nc_dir = '/Users/fanzhoukong/Documents/GitHub/Libgen_data/GT/nc_features'
from toolsets.precursor_matching import precursor_matching_alphapept
nc_matched = pd.DataFrame()
for mix in tqdm(std_list['Mix'].unique()):
    # break
    try:
        feature_filename = find_files(nc_dir, f"{'Mix_'+str(mix)}.csv")
        # print(len(feature_filename))
        # for filename in feature_filename:
        #     print(filename)
        # print(feature_filename[0])
        features_temp = read_in_alphapept(os.path.join(nc_dir, feature_filename[0]))
        nc_matched_temp = precursor_matching_alphapept(features_temp, std_list, mix_label=mix, adducts= std_list.columns[-4:], step=0.003, ppm=False)
        nc_matched = pd.concat([nc_matched, nc_matched_temp], axis=0)
    except:
        print("i do not found file for mix %s" %str(mix))
    # print((feature_filename[0]))


  # CSH_Mix_2.csv

100%|██████████| 15/15 [00:03<00:00,  4.95it/s]


In [41]:
nc_matched

,scan_number,precursor_mz,charge,ms1_intensity_ratio,ms1_precursor_intensity,retention_time,peaks,ms1_index,query_idx,reference_Mix,reference_Name,reference_Formula,reference_inchikey,reference_smiles,reference_adduct,reference_precursor_mz
714,2902,309.132446,1,0.754273,229004.0,5.754795,54.4532509\t0.0782516226\n60.8560219\t0.077488...,580,2320,1,Glutamine (L)_glycoside,C5H10N2O3,ZDXPYRJPNDTMRX-VKHMYHEASA-N,C(CC(=O)N)[C@@H](C(=O)O)N,[M+H]+,309.1297
39,315,235.166414,1,0.976972,5003663.0,0.625264,53.024212\t0.010527247\n56.049976\t0.02042501\...,63,251,1,Punctaporonin B,C15H24O3,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,[M-H2O+H]+,235.1693
75,384,235.166308,1,0.949520,184503584.0,0.760591,58.065536\t0.08439751\n59.073524\t0.094898134\...,77,306,1,Punctaporonin B,C15H24O3,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,[M-H2O+H]+,235.1693
87,417,235.166351,1,0.708066,54856968.0,0.824673,58.06562\t0.055201776\n59.07346\t0.065020993\n...,83,332,1,Punctaporonin B,C15H24O3,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,[M-H2O+H]+,235.1693
559,2394,235.170367,1,0.972644,33296404.0,4.740038,53.039066\t0.011861505\n55.047352\t0.024573449...,479,1914,1,Punctaporonin B,C15H24O3,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,[M-H2O+H]+,235.1693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,1763,343.122498,1,0.476687,220577.0,9.297420,66.7437897\t0.037596602\n70.0651169\t0.1083217...,1477,285,15,Isomaltulose,C12H22O11,PVXPPJIGRGXGCY-TZLCEDOOSA-N,C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)OC[C@@H]2...,[M+H]+,343.1235
5,197,391.137421,1,0.021722,57614.0,1.142228,57.070114\t0.038028698\n65.038757\t0.008740983...,183,13,15,Deoxyuridine_glycoside,C9H12N2O5,MXHRCPNRJAMMIM-SHYZEUOFSA-N,C1[C@@H]([C@H](O[C@H]1N2C=CC(=O)NC2=O)CO)O,[M+H]+,391.1352
63,1551,414.160714,1,1.000000,2858969.0,8.132979,56.049717\t0.031137727\n57.033817\t0.050027881...,1291,259,15,2'-deoxyadenosine_glycoside,C10H13N5O3,OLXZPDWKRNYJJZ-FSDSQADBSA-N,C1[C@H]([C@H](O[C@H]1N2C=NC3=C(N=CN=C32)N)CO)O,[M+H]+,414.1624
64,1584,414.160649,1,0.861566,663741.0,8.301914,56.049889\t0.042369954\n57.033775\t0.058208793...,1318,265,15,2'-deoxyadenosine_glycoside,C10H13N5O3,OLXZPDWKRNYJJZ-FSDSQADBSA-N,C1[C@H]([C@H](O[C@H]1N2C=NC3=C(N=CN=C32)N)CO)O,[M+H]+,414.1624


In [42]:
RT_matched = pd.DataFrame()
RT_unmatched = pd.DataFrame()
matched_rt = []
unmatched_rt = []
for index, row in tqdm(nc_matched.iterrows(), total=len(nc_matched)):
    data_temp = string_search(rt_list_checked, 'Substrate Name', row['reference_Name'])
    if len(data_temp)>0:
        rt = data_temp.iloc[0]['RT']
        if abs(row['retention_time']-rt)<=20/60:
            RT_matched=RT_matched.append(row)
            matched_rt.append(rt)
        else:
            RT_unmatched=RT_unmatched.append(row)
            unmatched_rt.append(rt)
    else:
        RT_matched=RT_matched.append(row)
        matched_rt.append('RT not found in list')
RT_matched['reference_RT']=matched_rt
RT_unmatched['reference_RT']=unmatched_rt

100%|██████████| 1593/1593 [00:01<00:00, 954.05it/s]


In [51]:
nc_matched

,scan_number,precursor_mz,charge,ms1_intensity_ratio,ms1_precursor_intensity,retention_time,peaks,ms1_index,query_idx,reference_Mix,reference_Name,reference_Formula,reference_inchikey,reference_smiles,reference_adduct,reference_precursor_mz
714,2902,309.132446,1,0.754273,229004.0,5.754795,54.4532509\t0.0782516226\n60.8560219\t0.077488...,580,2320,1,Glutamine (L)_glycoside,C5H10N2O3,ZDXPYRJPNDTMRX-VKHMYHEASA-N,C(CC(=O)N)[C@@H](C(=O)O)N,[M+H]+,309.1297
39,315,235.166414,1,0.976972,5003663.0,0.625264,53.024212\t0.010527247\n56.049976\t0.02042501\...,63,251,1,Punctaporonin B,C15H24O3,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,[M-H2O+H]+,235.1693
75,384,235.166308,1,0.949520,184503584.0,0.760591,58.065536\t0.08439751\n59.073524\t0.094898134\...,77,306,1,Punctaporonin B,C15H24O3,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,[M-H2O+H]+,235.1693
87,417,235.166351,1,0.708066,54856968.0,0.824673,58.06562\t0.055201776\n59.07346\t0.065020993\n...,83,332,1,Punctaporonin B,C15H24O3,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,[M-H2O+H]+,235.1693
559,2394,235.170367,1,0.972644,33296404.0,4.740038,53.039066\t0.011861505\n55.047352\t0.024573449...,479,1914,1,Punctaporonin B,C15H24O3,FCUGGFFHQXNXJN-MNRPPXDRSA-N,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,[M-H2O+H]+,235.1693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,1763,343.122498,1,0.476687,220577.0,9.297420,66.7437897\t0.037596602\n70.0651169\t0.1083217...,1477,285,15,Isomaltulose,C12H22O11,PVXPPJIGRGXGCY-TZLCEDOOSA-N,C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)OC[C@@H]2...,[M+H]+,343.1235
5,197,391.137421,1,0.021722,57614.0,1.142228,57.070114\t0.038028698\n65.038757\t0.008740983...,183,13,15,Deoxyuridine_glycoside,C9H12N2O5,MXHRCPNRJAMMIM-SHYZEUOFSA-N,C1[C@@H]([C@H](O[C@H]1N2C=CC(=O)NC2=O)CO)O,[M+H]+,391.1352
63,1551,414.160714,1,1.000000,2858969.0,8.132979,56.049717\t0.031137727\n57.033817\t0.050027881...,1291,259,15,2'-deoxyadenosine_glycoside,C10H13N5O3,OLXZPDWKRNYJJZ-FSDSQADBSA-N,C1[C@H]([C@H](O[C@H]1N2C=NC3=C(N=CN=C32)N)CO)O,[M+H]+,414.1624
64,1584,414.160649,1,0.861566,663741.0,8.301914,56.049889\t0.042369954\n57.033775\t0.058208793...,1318,265,15,2'-deoxyadenosine_glycoside,C10H13N5O3,OLXZPDWKRNYJJZ-FSDSQADBSA-N,C1[C@H]([C@H](O[C@H]1N2C=NC3=C(N=CN=C32)N)CO)O,[M+H]+,414.1624


In [45]:
file = pd.read_csv(os.path.join(source_dir, 'pet28.csv'))

In [55]:
string_search(nc_matched, 'reference_precursor_mz', 170.1539)

,scan_number,precursor_mz,charge,ms1_intensity_ratio,ms1_precursor_intensity,retention_time,peaks,ms1_index,query_idx,reference_Mix,reference_Name,reference_Formula,reference_inchikey,reference_smiles,reference_adduct,reference_precursor_mz
122,552,170.154631,1,0.96996,270588064.0,1.08379,55.0547752\t0.00632058876\n56.384407\t0.005840...,110,440,1,Lupinine,C10H19NO,HDVAWXXJVMJBAR-VHSXEESVSA-N,C1CCN2CCC[C@H]([C@H]2C1)CO,[M+H]+,170.1539


In [56]:
RT_matched_read = pd.read_csv(os.path.join(source_dir, "nc_matched_rt.csv"))

In [61]:
file.head(10)

,Count,index,reference_mix,scan_number,precursor_mz,ms1_precursor_intensity,retention_time,reference_substrate_name,reference_smiles,reference_inchikey,...,reference_adduct,reference_mix_enzyme,library-db,library-id,library-precursor_mz,library-adduct,library-charge,library-inchikey,library-identity_score,library-sugar_loss_intensity
0,1,1,1,1085,170.153781,349071584,2.123019,NaN,C1CCN2CCC[C@H]([C@H]2C1)CO,HDVAWXXJVMJBAR-VHSXEESVSA-N,...,[M+H]+,NaN,mona,VF-NPL-QEHF025268,170.15394,[M+H]+,1,HDVAWXXJVMJBAR,0.969917,0.0
1,2,2,1,1265,170.153814,148055440,2.477551,NaN,C1CCN2CCC[C@H]([C@H]2C1)CO,HDVAWXXJVMJBAR-VHSXEESVSA-N,...,[M+H]+,NaN,mona,VF-NPL-QEHF025268,170.15394,[M+H]+,1,HDVAWXXJVMJBAR,0.998418,0.0
2,3,3,1,1865,170.153832,6775184,3.660821,NaN,C1CCN2CCC[C@H]([C@H]2C1)CO,HDVAWXXJVMJBAR-VHSXEESVSA-N,...,[M+H]+,NaN,mona,NA000361,170.15394,[M+H]+,1,HDVAWXXJVMJBAR,0.782017,0.0
3,4,4,1,1955,170.153857,4366263,3.838859,NaN,C1CCN2CCC[C@H]([C@H]2C1)CO,HDVAWXXJVMJBAR-VHSXEESVSA-N,...,[M+H]+,NaN,mona,NA000330,170.15394,[M+H]+,1,HDVAWXXJVMJBAR,0.854183,0.0
4,5,5,1,2285,170.153796,3362243,4.492678,NaN,C1CCN2CCC[C@H]([C@H]2C1)CO,HDVAWXXJVMJBAR-VHSXEESVSA-N,...,[M+H]+,NaN,mona,NA000390,170.15394,[M+H]+,1,HDVAWXXJVMJBAR,0.784294,0.0
5,6,6,1,2836,170.153850,1990898,5.583856,NaN,C1CCN2CCC[C@H]([C@H]2C1)CO,HDVAWXXJVMJBAR-VHSXEESVSA-N,...,[M+H]+,NaN,mona,NA000390,170.15394,[M+H]+,1,HDVAWXXJVMJBAR,0.688898,0.0
6,7,7,1,2896,170.153869,1747917,5.703223,NaN,C1CCN2CCC[C@H]([C@H]2C1)CO,HDVAWXXJVMJBAR-VHSXEESVSA-N,...,[M+H]+,NaN,mona,NA000300,170.15394,[M+H]+,1,HDVAWXXJVMJBAR,0.613377,0.0
7,8,8,1,3618,170.153807,1573242,7.129379,NaN,C1CCN2CCC[C@H]([C@H]2C1)CO,HDVAWXXJVMJBAR-VHSXEESVSA-N,...,[M+H]+,NaN,mona,NA000390,170.15394,[M+H]+,1,HDVAWXXJVMJBAR,0.517842,0.0
8,9,9,1,9928,170.153788,1058385,19.721604,NaN,C1CCN2CCC[C@H]([C@H]2C1)CO,HDVAWXXJVMJBAR-VHSXEESVSA-N,...,[M+H]+,NaN,mona,NA000390,170.15394,[M+H]+,1,HDVAWXXJVMJBAR,0.548402,0.0
9,10,13,1,3183,481.315743,2684411,6.270718,NaN,C[C@]12CC[C@H]3C(=CC(=O)[C@H]4[C@@]3(C[C@@H]([...,NKDFYOWSKOHCCO-YPVLXUMRSA-N,...,[M+H]+,NaN,mona,VF-NPL-QEHF014043,481.31598,[M+H]+,1,NKDFYOWSKOHCCO,0.778138,0.0


In [60]:
string_search(RT_matched_read, 'reference_inchikey', 'HDVAWXXJVMJBAR-VHSXEESVSA-N')

,scan_number,precursor_mz,charge,ms1_intensity_ratio,ms1_precursor_intensity,retention_time,peaks,ms1_index,query_idx,reference_Mix,reference_Name,reference_Formula,reference_inchikey,reference_smiles,reference_adduct,reference_precursor_mz,reference_RT
1,2158,354.191101,1,0.878139,1386332.0,4.269993,51.204578\t0.0071129836\n53.402023\t0.00679022...,432,1725,1,Lupinine_glycoside,C10H19NO,HDVAWXXJVMJBAR-VHSXEESVSA-N,C1CCN2CCC[C@H]([C@H]2C1)CO,[M+Na]+,354.1892,RT not found in list


In [57]:
RT_matched_read

,scan_number,precursor_mz,charge,ms1_intensity_ratio,ms1_precursor_intensity,retention_time,peaks,ms1_index,query_idx,reference_Mix,reference_Name,reference_Formula,reference_inchikey,reference_smiles,reference_adduct,reference_precursor_mz,reference_RT
0,2902,309.132446,1,0.754273,229004.0,5.754795,54.4532509\t0.0782516226\n60.8560219\t0.077488...,580,2320,1,Glutamine (L)_glycoside,C5H10N2O3,ZDXPYRJPNDTMRX-VKHMYHEASA-N,C(CC(=O)N)[C@@H](C(=O)O)N,[M+H]+,309.1297,RT not found in list
1,2158,354.191101,1,0.878139,1386332.0,4.269993,51.204578\t0.0071129836\n53.402023\t0.00679022...,432,1725,1,Lupinine_glycoside,C10H19NO,HDVAWXXJVMJBAR-VHSXEESVSA-N,C1CCN2CCC[C@H]([C@H]2C1)CO,[M+Na]+,354.1892,RT not found in list
2,3427,451.101688,1,0.926871,2411171.0,6.806105,50.634056\t0.11018493\n70.00631\t0.13219266\n7...,685,2740,1,Rutilantinone,C22H20O9,RWCVSDKDFSVZNF-KRYGIPSASA-N,CC[C@]1(C[C@@H](C2=C(C3=C(C=C2[C@H]1C(=O)OC)C(...,[M+Na]+,451.1000,RT not found in list
3,2109,643.369202,1,0.523223,148181.0,4.171581,70.0655289\t0.353958696\n74.8802719\t0.0423589...,422,1686,1,CRUSTECDYSONE_glycoside,C27H44O7,NKDFYOWSKOHCCO-YPVLXUMRSA-N,C[C@]12CC[C@H]3C(=CC(=O)[C@H]4[C@@]3(C[C@@H]([...,[M+H]+,643.3693,RT not found in list
4,3203,505.245479,1,0.959315,1897799.0,6.358225,50.3740997\t0.0131859099\n52.0901489\t0.016589...,641,2561,1,Cortisone_glycoside,C21H28O5,MFYSYFVPBJMHGN-ZPOLXVRWSA-N,C[C@]12CCC(=O)C=C1CC[C@@H]3[C@@H]2C(=O)C[C@]4(...,[M-H2O+H]+,505.2437,RT not found in list
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,1237,300.143989,1,0.649006,2300005.0,6.444129,53.03899\t0.0155733433\n56.0497131\t0.00721621...,1020,215,15,TYRAMINE_glycoside,C8H11NO,DZGWFCGJZKJUFP-UHFFFAOYSA-N,C1=CC(=CC=C1CCN)O,[M+H]+,300.1446,RT not found in list
309,1269,300.144228,1,0.817279,3535120.0,6.609112,53.038933\t0.019467197\n69.054306\t0.006381815...,1047,221,15,TYRAMINE_glycoside,C8H11NO,DZGWFCGJZKJUFP-UHFFFAOYSA-N,C1=CC(=CC=C1CCN)O,[M+H]+,300.1446,RT not found in list
310,1304,300.144226,1,0.708576,12851130.0,6.778391,53.038963\t0.019324495\n69.033516\t0.006680520...,1074,229,15,TYRAMINE_glycoside,C8H11NO,DZGWFCGJZKJUFP-UHFFFAOYSA-N,C1=CC(=CC=C1CCN)O,[M+H]+,300.1446,RT not found in list
311,1551,414.160714,1,1.000000,2858969.0,8.132979,56.049717\t0.031137727\n57.033817\t0.050027881...,1291,259,15,2'-deoxyadenosine_glycoside,C10H13N5O3,OLXZPDWKRNYJJZ-FSDSQADBSA-N,C1[C@H]([C@H](O[C@H]1N2C=NC3=C(N=CN=C32)N)CO)O,[M+H]+,414.1624,RT not found in list


In [24]:
RT_matched.to_csv(os.path.join(source_dir, "nc_matched_rt.csv"), index= False)
RT_unmatched.to_csv(os.path.join(source_dir, "nc_unmatched_rt.csv"), index= False)

In [86]:
nc_matched.to_csv(os.path.join(source_dir, 'pet_matched.csv'), index=False)

In [58]:
len(substrate_matched['reference_smiles'].unique())

385

# deduplication for msms matched files

In [3]:
from toolsets.deduplication import _find_seed_rt
def iterative_find_rt_match(compound):
    seed = _find_seed_rt(compound,rt_tolerance = 10/60, iterative=True)
    rt_matched = string_search(compound,'rt_match_id',seed).iloc[0]['retention_time']
    matched = num_search(compound,'retention_time', rt_matched, direction='between', step = 10/60, inclusion=True)
    matched['seed_found']=seed
    unmatched = compound[~compound['rt_match_id'].isin(matched['rt_match_id'])]
    return (matched,unmatched)
def find_unique(data):
    data['rt_match_id']= np.arange(len(data))
    matched_all = pd.DataFrame()
    for mix in tqdm(data['File name'].unique()):
        data_mix = string_search(data, 'File name',mix)
        for product in data_mix['Product name'].unique():
            compound = string_search(data_mix,'Product name',product)
            while len(compound)>=1:
                matched, compound = iterative_find_rt_match(compound)
                matched_all=matched_all.append(matched)
            # return (matched)
            # return (compound)
            # if len(compound)>0:
            #     compound['seed_found']=compound.iloc[0]['rt_match_id']
            #     matched_all=matched_all.append(compound)
    return (matched_all)
            # if len(compound)>3:
            #     return (compound)

In [6]:
data = pd.read_csv(os.path.join(source_dir, 'data_only.csv'))
data_true = string_search(data, 'Library match', True)
data_true.reset_index(inplace=True, drop=True)
data_false = string_search(data, 'Library match', False)
data_false.reset_index(inplace=True, drop=True)

In [7]:
data.head()

,Output type,precursor_mz,ms1_precursor_intensity,retention_time,Product name,Aglycone name,Aglycone RT,RT shift (product-substrate)RP or (substrate-product)NP,Aglycone smiles,Aglycone InChikey,...,library-inchikey,library-name,library-identity_score,Product ion intensity (-120.042 loss),Product ion intensity (-162.0533 loss),Product ion intensity (-324.1066 loss),Product ion intensity (-90.032 loss),inchikey,smiles,bond_difference
0,162 shift,317.1242,7120000.0,3.42,"2,6-Dihydroxy-4-Methoxytoluene_glycoside","2,6-Dihydroxy-4-Methoxytoluene",4.4,-0.98,CC1=C(C=C(C=C1O)OC)O,YPIDZFLQKNEKRA-UHFFFAOYSA-N,...,XQDNFAMOIPNVES,"3,5-DIMETHOXYPHENOL",0.445285,0.0,0.192825,0.0,0.0,XQDNFAMOIPNVES,COC1=CC(=CC(=C1)O)OC,1.0
1,162 shift,419.1334,16000000.0,5.67,"3,4-Didesmethyl-5-Deshydroxy-3'-Ethoxyscleroin...","3,4-Didesmethyl-5-Deshydroxy-3'-Ethoxyscleroin",8.47,-2.8,CCOC1=CC=CC(=C1)C(=O)C2=C(C(=C(C=C2)O)O)O,IPLQLQEWXCSIX-UHFFFAOYSA-N,...,DXDRHHKMWQZJHT,isoliquiritigenin,0.36066824,0.0,0.013065,0.0,0.0,DXDRHHKMWQZJHT,C1=CC(=CC=C1C=CC(=O)C2=C(C=C(C=C2)O)O)O,3.5
2,162 shift,437.1438,5130000.0,5.55,"3,4-Didesmethyl-5-Deshydroxy-3'-Ethoxyscleroin...","3,4-Didesmethyl-5-Deshydroxy-3'-Ethoxyscleroin",8.47,-2.92,CCOC1=CC=CC(=C1)C(=O)C2=C(C(=C(C=C2)O)O)O,IPLQLQEWXCSIX-UHFFFAOYSA-N,...,VYIBOCHDJVESIU,"2,3,4-TRIHYDROXY-4'-ETHOXYBENZOPHENONE",0.70524913,0.0,0.000000,0.0,0.0,VYIBOCHDJVESIU,CCOC1=CC=C(C=C1)C(=O)C2=C(C(=C(C=C2)O)O)O,1.0
3,162 shift,537.3416,10000000.0,7.70,Alpha-Hydroxydeoxycholic Acid_glycoside,Alpha-Hydroxydeoxycholic Acid,15.91,-8.21,C[C@H](CCC(=O)O)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,DGABKXLVXPYZII-SIBKNCMHSA-N,...,KIQFUORWRVZTHT,3-Ketocholanic Acid,0.6399134,0.0,0.000000,0.0,0.0,KIQFUORWRVZTHT,CC(CCC(=O)O)C1CCC2C1(CCC3C2CCC4C3(CCC(=O)C4)C)C,1.5
4,162 shift,555.3520,38300000.0,7.67,Alpha-Hydroxydeoxycholic Acid_glycoside,Alpha-Hydroxydeoxycholic Acid,15.91,-8.24,C[C@H](CCC(=O)O)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,DGABKXLVXPYZII-SIBKNCMHSA-N,...,KXGVEGMKQFWNSR,Deoxycholic Acid,0.7789851,0.0,0.000000,0.0,0.0,KXGVEGMKQFWNSR,CC(CCC(=O)O)C1CCC2C1(C(CC3C2CCC4C3(CCC(C4)O)C)O)C,1.0


In [8]:
data_true_seed = find_unique(data_true)
data_false_seed = find_unique(data_false)

100%|██████████| 1182/1182 [00:13<00:00, 89.42it/s] 


In [9]:
data_true_seed.head()

,Output type,precursor_mz,ms1_precursor_intensity,retention_time,Product name,Aglycone name,Aglycone RT,RT shift (product-substrate)RP or (substrate-product)NP,Aglycone smiles,Aglycone InChikey,...,Product ion intensity (-120.042 loss),Product ion intensity (-162.0533 loss),Product ion intensity (-324.1066 loss),Product ion intensity (-90.032 loss),inchikey,smiles,bond_difference,rt_match_id,hits_rt,seed_found
0,162 shift,470.3108,5610000.0,8.20,Androsterone_glycoside,Androsterone,10.21,-2.01,C[C@]12CC[C@H](C[C@@H]1CC[C@@H]3[C@@H]2CC[C@]4...,QGXBDMJGAMFCBF-HLUDHZFRSA-N,...,0.0,0.0,0.0,0.0,QGXBDMJGAMFCBF,CC12CCC(CC1CCC3C2CCC4(C3CCC4=O)C)O,0.0,0,1,0
1,162 shift,453.2844,677000.0,8.15,Androsterone_glycoside,Androsterone,10.21,-2.06,C[C@]12CC[C@H](C[C@@H]1CC[C@@H]3[C@@H]2CC[C@]4...,QGXBDMJGAMFCBF-HLUDHZFRSA-N,...,0.0,0.0,0.0,0.0,QGXBDMJGAMFCBF,CC12CCC(CC1CCC3C2CCC4(C3CCC4=O)C)O,0.0,1,1,0
2977,324 shift,632.3634,3520000.0,7.92,Androsterone_diglycoside,Androsterone,10.21,-2.29,C[C@]12CC[C@H](C[C@@H]1CC[C@@H]3[C@@H]2CC[C@]4...,QGXBDMJGAMFCBF-HLUDHZFRSA-N,...,0.0,0.0,0.0,0.0,QGXBDMJGAMFCBF,CC12CCC(CC1CCC3C2CCC4(C3CCC4=O)C)O,0.0,2977,0,2977
2978,324 shift,632.3633,17400000.0,7.29,Androsterone_diglycoside,Androsterone,10.21,-2.92,C[C@]12CC[C@H](C[C@@H]1CC[C@@H]3[C@@H]2CC[C@]4...,QGXBDMJGAMFCBF-HLUDHZFRSA-N,...,0.0,0.0,0.0,0.0,QGXBDMJGAMFCBF,CC12CCC(CC1CCC3C2CCC4(C3CCC4=O)C)O,0.0,2978,0,2978
3749,162 shift,468.2584,8780000.0,8.17,CAPSAICIN_glycoside,CAPSAICIN,9.75,-1.58,CC(C)/C=C/CCCCC(=O)NCC1=CC(=C(C=C1)O)OC,YKPUWZUDDOIDPM-SOFGYWHQSA-N,...,0.0,0.0,0.0,0.0,YKPUWZUDDOIDPM,CC(C)C=CCCCCC(=O)NCC1=CC(=C(C=C1)O)OC,0.0,3749,0,3749


In [180]:
# data_true_seed = find_unique(data_true)
# data_false_seed = find_unique(data_false)

100%|██████████| 1182/1182 [00:12<00:00, 91.15it/s] 


In [10]:
from toolsets.spectra_operations import weighted_average_spectra
for dataset in [data_true_seed,data_false_seed]:
    # wa_spctra = []
    # wa_rts = []
    dataset['key']=dataset['seed_found'].astype(str)+dataset['Expected precursor_adduct']
    dataset.sort_values(by = ['key','ms1_precursor_intensity'], ascending=False, inplace=True)
    dataset.drop_duplicates('key', keep='first', inplace=True)


In [12]:
data_true_seed.reset_index(inplace=True, drop=True)
data_false_seed.reset_index(inplace=True, drop=True)

In [13]:
data_true_seed.to_csv(os.path.join(source_dir, 'msms_matched_deduplicated.csv'), index = False)
data_false_seed.to_csv(os.path.join(source_dir, 'msms_unmatched_deduplicated.csv'), index = False)

In [188]:
# from toolsets.helpers import split_unique_duplicates
# # print("first round of unique keys confirming...")
# matched_confirmed, matched_unconfirmed = split_unique_duplicates(matched)
# len(matched['reference_Substrate Name'].unique())
# len(matched)
# from toolsets.helpers import match_by_rt
# print("second round of confirming via rt...")
# matched_confirmed_rt, matched_unconfirmed_rt =match_by_rt(matched_confirmed, matched_unconfirmed)
# from toolsets.helpers import deduplicate
# print("start deduplicating of unconfirmed matches...")
# deduplicated = deduplicate(matched_unconfirmed_rt)
# print("deduplicating of unconfirmed matches done")
# result = pd.concat([matched_confirmed_rt, deduplicated], axis=0)
# result.reset_index(inplace=True, drop= True)
# len(result['reference_Substrate Name'].unique())
# len(matched['reference_Substrate Name'].unique())
# result.to_csv(os.path.join(source_dir, "deduplicated_result.csv"), index = False)